In [1]:
import numpy as np
np.set_printoptions(precision = 4, linewidth = 132)

# E = Modulo_elastico ;  S = seccion (considerando todas iguales)
# MC = Matriz conectividad ; MN = Matriz nodos 

Elementos = 5
MC = [(0,1),(0,3),(1,3),(1,2),(2,3)]
MN = [(0,0),(8,4),(16,0),(8,0)]
E = 210E9
SECCIONES = [0.001, 0.001, 0.002, 0.001, 0.001]

#1 es condicion de fuerza, 2 es condicion de desplazamiento

Vinculos = [(0,0), # x nodo 0
            (0,0), # y nodo 0 
            (1,0), # x nodo 1
            (1,0), # y nodo 1 
            (1,0), # x nodo 2
            (0,0), # y nodo 2            
            (1,0), # x nodo 3
            (1,-20E3)] # y nodo 3 

#MC = [(0,1),(1,2),(0,2)] #Matriz_conectividad (MC)
#MN = [(0,0),(1,0),(1,1)] #Matriz_nodos (MN)

#AUXILIARES

n = len(MN) 
Matriz_global = np.zeros([n*2,n*2])
Matrices_loc = []
Fuerza = []
Nodos = []
Desplazamientos_f = np.zeros(n*2)
Tensiones_elementos = []
Desplazamientos_local = np.zeros(Elementos-1)

In [2]:
for i in range(Elementos): # si range(n) -- 0 1 2 3 pero yo tengo 0 1 2 3 4 elementos ahora no es general pues si cargo el otro ejercicio tendre un elemento de mas
    
     Sec = SECCIONES[i]
        
     p,j = MC[i]
    
     p1,p2 = MN[p]
        
     j1,j2 = MN[j]
    
     Largo = np.sqrt((j1-p1)**2 + (j2-p2)**2) 
        
     angulo = np.arctan2(j2-p2,j1-p1) #angulo con cuadrante definido
    
     C = np.cos(angulo) 
        
     S = np.sin(angulo)  
    
     matriz = np.array([[C**2,C*S,-C**2,-C*S],[C*S,S**2,-C*S,-S**2],[-C**2,-C*S,C**2,C*S],[-C*S,-S**2,C*S,S**2]]).round(8)
        
     K = (E*Sec)/Largo
    
     componente = K*matriz        
        
     Matrices_loc.append(componente)  
    
     A_G = np.array([p*2,p*2+1],dtype = int)
     B_G = np.array([j*2,j*2+1],dtype = int) 
    
     A_L = np.array([0,1],dtype = int)
     B_L = np.array([2,3],dtype = int)

    
     Matriz_global[np.ix_(A_G,A_G)] +=  componente [np.ix_(A_L,A_L)]
     Matriz_global[np.ix_(B_G,B_G)] +=  componente [np.ix_(B_L,B_L)]
     Matriz_global[np.ix_(A_G,B_G)] +=  componente [np.ix_(A_L,B_L)]
     Matriz_global[np.ix_(B_G,A_G)] +=  componente [np.ix_(B_L,A_L)]
    
print(Matriz_global)

[[ 4.5033e+07  9.3915e+06 -1.8783e+07 -9.3915e+06  0.0000e+00  0.0000e+00 -2.6250e+07  0.0000e+00]
 [ 9.3915e+06  4.6957e+06 -9.3915e+06 -4.6957e+06  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [-1.8783e+07 -9.3915e+06  3.7566e+07  0.0000e+00 -1.8783e+07  9.3915e+06  0.0000e+00  0.0000e+00]
 [-9.3915e+06 -4.6957e+06  0.0000e+00  1.1439e+08  9.3915e+06 -4.6957e+06  0.0000e+00 -1.0500e+08]
 [ 0.0000e+00  0.0000e+00 -1.8783e+07  9.3915e+06  4.5033e+07 -9.3915e+06 -2.6250e+07  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  9.3915e+06 -4.6957e+06 -9.3915e+06  4.6957e+06  0.0000e+00  0.0000e+00]
 [-2.6250e+07  0.0000e+00  0.0000e+00  0.0000e+00 -2.6250e+07  0.0000e+00  5.2500e+07  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 -1.0500e+08  0.0000e+00  0.0000e+00  0.0000e+00  1.0500e+08]]


In [3]:


for i in range(len(Vinculos)):
    tipo,valor = Vinculos[i]  
    if tipo == 1:
        Fuerza.append(valor)
        Nodos.append(i)
    if tipo == 0:
        Desplazamientos_f[i] = valor

Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
    Desplazamientos_f[i] = Desplazamientos_parcial [o]
    o += 1
  

Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
print(Fuerzas_)
print()
print(Desplazamientos_f)

[ 0.0000e+00  1.0000e+04  0.0000e+00  3.4561e-11  1.4552e-11  1.0000e+04  0.0000e+00 -2.0000e+04]

[ 0.      0.      0.0008 -0.0037  0.0015  0.      0.0008 -0.0038]


In [4]:

l = 0
for i in range(Elementos):
       p,j = MC[i]
       Desplazamientos_local [0] = Desplazamientos_f[2*p]
       Desplazamientos_local [1] = Desplazamientos_f[2*p+1]
       Desplazamientos_local [2] = Desplazamientos_f[2*j]
       Desplazamientos_local [3] = Desplazamientos_f[2*j+1]
       Desplazamientos_local = Desplazamientos_local.reshape(-1,1)
       Matriz_loc = Matrices_loc[l]
       l += 1
       Tensiones_elementos.append(np.dot(Matriz_loc, Desplazamientos_local))

In [ ]:
#      A_G = np.array([p*2,p*2+1],dtype = int)
        
#      B_G = np.array([j*2,j*2+1],dtype = int) 


        
#      Matriz_global[np.ix_(A_G,A_G)] +=  componente
#      Matriz_global[np.ix_(B_G,B_G)] +=  componente
#      Matriz_global[np.ix_(A_G,B_G)] +=  -componente
#      Matriz_global[np.ix_(B_G,A_G)] +=  -componente 

             